In [115]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

In [116]:
cash = pd.read_csv('../project_dataset/cash_request.csv')

In [117]:
cash

,id,amount,status,created_at,updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update
0,5,100.0,rejected,2019-12-10 19:05:21.596873+00,2019-12-11 16:47:42.40783+00,804.0,2019-12-11 16:47:42.405646+00,NaN,2020-01-09 19:05:21.596363+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
1,70,100.0,rejected,2019-12-10 19:50:12.34778+00,2019-12-11 14:24:22.900054+00,231.0,2019-12-11 14:24:22.897988+00,NaN,2020-01-09 19:50:12.34778+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
2,7,100.0,rejected,2019-12-10 19:13:35.82546+00,2019-12-11 09:46:59.779773+00,191.0,2019-12-11 09:46:59.777728+00,NaN,2020-01-09 19:13:35.825041+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
3,10,99.0,rejected,2019-12-10 19:16:10.880172+00,2019-12-18 14:26:18.136163+00,761.0,2019-12-18 14:26:18.128407+00,NaN,2020-01-09 19:16:10.879606+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
4,1594,100.0,rejected,2020-05-06 09:59:38.877376+00,2020-05-07 09:21:55.34008+00,7686.0,2020-05-07 09:21:55.320193+00,NaN,2020-06-05 22:00:00+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23965,20616,100.0,money_back,2020-10-12 13:54:11.686225+00,2021-02-06 20:17:49.292493+00,13681.0,NaN,NaN,2021-02-06 11:00:00+00,2020-10-13,2021-02-06 20:17:49.257521+00,instant,2020-10-12 13:54:24.352856+00,NaN,NaN,NaN
23966,25243,50.0,money_back,2020-10-27 14:41:25.73491+00,2020-12-18 13:15:40.843946+00,NaN,NaN,30367.0,2020-11-03 22:00:00+00,2020-10-28,2020-12-01 13:26:53.787672+00,instant,2020-10-27 14:41:57.901946+00,completed,2020-11-12 23:20:41.928788+00,2020-12-01 13:26:53.815504+00
23967,22357,100.0,money_back,2020-10-20 07:58:04.006937+00,2021-02-05 12:19:30.656816+00,82122.0,NaN,NaN,2021-02-05 11:00:00+00,2020-10-21,2021-02-05 12:19:30.626289+00,instant,2020-10-20 07:58:14.171553+00,NaN,NaN,NaN
23968,20256,100.0,money_back,2020-10-10 05:40:55.700422+00,2021-02-05 13:14:19.707627+00,64517.0,NaN,NaN,2021-02-05 11:00:00+00,2020-10-12,2021-02-05 13:14:19.689906+00,instant,2020-10-10 05:41:23.368363+00,NaN,NaN,NaN


In [118]:
cash['created_at'] = pd.to_datetime(cash['created_at'])
cash['updated_at'] = pd.to_datetime(cash['updated_at'])
cash['moderated_at'] = pd.to_datetime(cash['moderated_at'],unit='ns')
cash['reimbursement_date'] = pd.to_datetime(cash['reimbursement_date'],unit='ns')
cash['cash_request_received_date'] = pd.to_datetime(cash['cash_request_received_date'],unit='ns')
cash['money_back_date'] = pd.to_datetime(cash['money_back_date'],unit='ns')
cash['send_at'] = pd.to_datetime(cash['send_at'],unit='ns')
cash['reco_creation'] = pd.to_datetime(cash['reco_creation'],unit='ns')
cash['reco_last_update'] = pd.to_datetime(cash['reco_last_update'],unit='ns')

In [119]:
prestamos_devueltos = cash[cash['money_back_date'].notna()]
prestamos_devueltos = prestamos_devueltos[prestamos_devueltos['status'] == "money_back"]
prestamos_devueltos = prestamos_devueltos[prestamos_devueltos['money_back_date'] >= prestamos_devueltos['created_at']]
prestamos_devueltos['days_reimbursement'] = (prestamos_devueltos['money_back_date'] - prestamos_devueltos['created_at']).dt.days

print(prestamos_devueltos['days_reimbursement'].max())
print(prestamos_devueltos['days_reimbursement'].min())
print(prestamos_devueltos['days_reimbursement'].mean())

375
0
32.0251758607923


In [120]:
# Calcular el tiempo medio de days_reimbursement para cada valor de amount
mean_days_reimbursement = cash.groupby('amount')['days_reimbursement'].mean().reset_index()

# Crear un mapa de calor
plt.figure(figsize=(10, 6))
heatmap = sns.heatmap(mean_days_reimbursement.pivot_table(index='days_reimbursement', columns='amount', values='days_reimbursement'), cmap='viridis')
heatmap.set_title('Tiempo medio de days_reimbursement por amount')
heatmap.set_xlabel('Amount')
heatmap.set_ylabel('Tiempo medio de days_reimbursement')
plt.show()

KeyError: 'Column not found: days_reimbursement'

In [ ]:
prestamos_devueltos.info()


heatmap_data = prestamos_devueltos[['days_reimbursement', 'cohorte' 'amount']].dropna()  # Eliminar filas con valores NaN
heatmap_data_group =  heatmap_data.groupby(["days_reimbursement", "cohorte"])["amount"].mean()
heatmap_data_group["cohorte"] = heatmap_data_group["days_reimbursement"]


In [ ]:
prestamos_devueltos['cohort_month'] = prestamos_devueltos['created_at'].dt.to_period('M')

# Calcular la cantidad de usuarios únicos en cada cohorte
cohort_data = prestamos_devueltos.groupby(['cohort_month']).agg({'user_id': 'nunique'})

# Calcular la tasa de retención
cohort_data['retention_rate'] = cohort_data['user_id'] / cohort_data['user_id'].iloc[0]

# Visualizar la tasa de retención
plt.figure(figsize=(10, 6))
plt.plot(cohort_data.index.astype(str), cohort_data['retention_rate'], marker='o')
plt.title('Tasa de Retención por Cohorte')
plt.xlabel('Mes de Cohorte')
plt.ylabel('Tasa de Retención')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Extraer el mes de la columna 'created_at' para definir las cohortes
prestamos_devueltos['cohort_month'] = prestamos_devueltos['created_at'].dt.to_period('M')

# Calcular la cantidad de usuarios únicos en cada cohorte
cohort_data = prestamos_devueltos.groupby(['cohort_month']).agg({'user_id': 'nunique'})

# Calcular la tasa de retención
cohort_data['retention_rate'] = cohort_data['user_id'] / cohort_data['user_id'].iloc[0]

# Convertir los meses de las cohortes a strings
cohort_data.reset_index(inplace=True)
cohort_data['cohort_month'] = cohort_data['cohort_month'].astype(str)

# Crear un mapa de calor de la tasa de retención
plt.figure(figsize=(12, 8))
heatmap_data = cohort_data.pivot(index='cohort_month', columns='cohort_month', values='retention_rate')
sns.heatmap(heatmap_data, annot=True, fmt=".2%", cmap="YlGnBu", cbar_kws={'format': '%.0f%%'})
plt.title('Tasa de Retención por Cohorte')
plt.xlabel('Cohorte')
plt.ylabel('Cohorte')
plt.show()